<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/Multimodal/Multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram K

### Setup

In [ ]:
!pip install transformers

!pip install pytesseract
# !pip install pdf2image
!apt-get install poppler-utils 
!apt install tesseract-ocr
# !pip install keras_nlp

### Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import re

import os
from pathlib import Path

import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling1D, BatchNormalization, Embedding, Bidirectional, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# import keras_nlp

from sklearn.model_selection import train_test_split

import spacy
from unicodedata import normalize

from tqdm import tqdm

import pytesseract
from PIL import Image

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

import plotly.express as px

### Gathering Dataset

In [ ]:
!unzip /content/drive/MyDrive/Deceptive-Research/dataset.zip

### Global Variables

In [3]:
img_shape = (224, 224)
deceptive_path = Path("/content/deceptive")
normal_path = Path("/content/Ads/")
uniq_labels = ["Non Deceptive", "Deceptive"]

### Cleaning the Data

In [4]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [5]:
def clean_text(text):

    text = normalize("NFKD", text) #Normalization

    text = re.sub(r"[^\w\s]","", text) #Remove Punc

    text = " ".join([token.lemma_ for token in nlp(text) if not token.is_stop])

    text = re.sub("\s+", " ", text)

    text = text.strip()

    return text

In [6]:
def clean_img(fname):

  img = cv2.imread(fname)

  img = cv2.resize(img, img_shape) 

  # Normalization
  img = img/255.0

  return img

### OCR - Image to Text

In [7]:
def get_data(image):

  txt = pytesseract.image_to_string(Image.open(image), lang="eng")
  txt = re.sub("[\n]{2,}", "\t\t", txt)
  txt = re.sub("\n", "", txt)
  txt = re.sub("\t\t", "\n", txt)

  return txt

### Image Extraction

In [8]:
def extract_image(path, target):

  X1 = []
  X2 = []
  y = []

  for img in os.listdir(path):

    _, tail = os.path.splitext(img)

    if tail in [".jpg", ".jpeg", ".png"]:

      try:
        
        fname = f"{path}/{img}"

        # OCR - Image to Text        
        text = get_data(fname)

        # Cleaning the text
        text = clean_text(text)

        if not text:
          text = "No Information"
          print(fname)

        # Cleaning the Image
        img = clean_img(fname)

        X1.extend([img])
        X2.extend([text])
        y.extend([target])

      except Exception as e: print(f"Exception: {e}")

  return X1, X2, y

In [ ]:
X1, X2, y = extract_image(deceptive_path, 1)

In [10]:
for i in os.listdir(normal_path):
  X1_t, X2_t, y_t = extract_image(f"{normal_path}/{i}", 0)
  X1.extend(X1_t)
  X2.extend(X2_t)
  y.extend(y_t)

/content/Ads/15/13.png
/content/Ads/3/11.png


In [11]:
X1 = np.array(X1)
# X2 = np.array(X2)

y = np.array(y, dtype="float32")

In [12]:
X1.shape, y.shape

((392, 224, 224, 3), (392,))

In [13]:
len_X2 = [len(x.split()) for x in X2]

In [14]:
px.box(len_X2)

### Tokenization

In [15]:
checkpoint = "bert-base-uncased"
# checkpoint = "gpt2"
sequence_length = 100

def tokenize(samples):

  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  if checkpoint == "gpt2" and tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

  tokens = tokenizer(
      samples,
      max_length=sequence_length,
      truncation=True,
      padding="max_length",
      add_special_tokens=True,
      return_tensors="np"
  )

  return {"input_ids": tokens["input_ids"].tolist(), "attention_mask": tokens["attention_mask"].tolist()}

In [16]:
X_tokenized = pd.DataFrame(tokenize(X2), columns=["input_ids", "attention_mask"])

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

### Splitting Data into Train/Test

In [17]:
X_tokenized_train, X_tokenized_test, X_img_train, X_img_test, y_train, y_test = train_test_split(X_tokenized, X1, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

In [18]:
X_tokenized_train

,input_ids,attention_mask
217,"[101, 4770, 2795, 5096, 7479, 10760, 16869, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
308,"[101, 2489, 7829, 7479, 13278, 9692, 9006, 248...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
103,"[101, 4497, 8913, 1053, 2669, 2080, 1052, 1034...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
15,"[101, 2053, 2592, 102, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
230,"[101, 2184, 12849, 10264, 2102, 2192, 5906, 37...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."
...,...,...
100,"[101, 9686, 17443, 1023, 2243, 2290, 24707, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
273,"[101, 3046, 12882, 2382, 2154, 2489, 3711, 231...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
276,"[101, 3338, 2919, 3143, 2186, 11387, 16068, 23...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
277,"[101, 10036, 22477, 14154, 9938, 2860, 2860, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


### Unzipping the IDs and Masks 

In [19]:
unzip_x = lambda x: [np.vstack(x["input_ids"]), np.vstack(x["attention_mask"])]

X_tokenized_train, X_tokenized_test = unzip_x(X_tokenized_train), unzip_x(X_tokenized_test)

In [20]:
X_train = [X_img_train, X_tokenized_train]
X_test = [X_img_test, X_tokenized_test]

### Building the model

In [25]:
def build_model(img_shape, targets, checkpoint, sequence_length):

  # Loading the pre-trained Resnet model
  base_model_img = tf.keras.applications.ResNet50(input_shape=img_shape, include_top=False, weights="imagenet")
  
  # Loading the pre-trained BERT model
  base_model_txt = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

  # Freezing the base model
  base_model_img.trainable = False

  input_ids = Input(shape=(sequence_length,), name="input_ids", dtype="int32")
  attenion_mask = Input(shape=(sequence_length,), name="attention_mask", dtype="int32")

  if checkpoint == "gpt2": x1 = base_model_txt.transformer(input_ids, attention_mask=attenion_mask)[0]
  else: x1 = base_model_txt.bert(input_ids, attention_mask=attenion_mask)[1]

  # Defining the custom head for our neural network
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_img.output)

  x1 = Flatten()(x1)

  # Concat the results

  concat_vec = tf.keras.layers.concatenate([x1, global_average_layer], name="Concat")
  # x = tf.concat([x1, global_average_layer], axis=-1, name="Concat")

  # # x = tf.keras.layers.Reshape(target_shape=((1, x.shape) + (1,)), name="Reshaping")(x)
  # expanded_concat = tf.keras.layers.Reshape((concat_vec.shape[1], 1))(concat_vec)

  # print(expanded_concat)
  
  # # Encoder Layer
  # x = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)(expanded_concat)
  # x = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)(x)

  # # Squeeze
  # x = tf.squeeze(x, axis=-1)  

  # Output layer
  units = len(np.unique(targets))

  if units > 2:
      activation = "softmax"
      loss = "sparse_categorical_crossentropy"
  
  else:
      activation = "sigmoid"
      loss = "binary_crossentropy"
      units = units - 1
  
  outputs = Dense(units=units, activation=activation)(concat_vec)

  model = Model(inputs=[base_model_img.input, input_ids, attenion_mask], outputs=outputs)

  # Compiling the model
  model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

  # Model Architecture Export
  tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, 
                            show_dtype=True, show_layer_names=True, rankdir='TB',
                            expand_nested=True, dpi=300, layer_range=None, 
                            show_layer_activations=True)

  
  return model

In [26]:
model = build_model((224, 224, 3), y, checkpoint, sequence_length)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.437086 to fit



### Callbacks

In [27]:
def callbacks() -> list:

    run_name = "run 1"
    save_path = Path("/models")
    os.makedirs(save_path/"logs", exist_ok=True)
    
    checkpoint = ModelCheckpoint(save_path, monitor="val_loss", save_best_only=True, 
                                                    verbose=1)

    earlystopping = EarlyStopping(monitor="val_loss", verbose=1, restore_best_weights = True,
                                                    patience=5)

    logger = TensorBoard(save_path/"logs"/run_name, histogram_freq=2, write_graph=True, write_images=True)

    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2, verbose=1,
                                         min_delta=0.0001, cooldown=0, min_lr=0)
    
    return [checkpoint, earlystopping, lr, logger]

In [28]:
#  !sudo rm -rf /content/drive/MyDrive/Deceptive-Research/models

### Model Training

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=8, callbacks=callbacks())

Epoch 1/100
40/40 [==============================] - ETA: 0s - loss: 0.7696 - accuracy: 0.7061
Epoch 1: val_loss improved from inf to 0.86203, saving model to /models


40/40 [==============================] - 137s 3s/step - loss: 0.7696 - accuracy: 0.7061 - val_loss: 0.8620 - val_accuracy: 0.2405 - lr: 0.0010
Epoch 2/100
40/40 [==============================] - ETA: 0s - loss: 0.5682 - accuracy: 0.7252
Epoch 2: val_loss improved from 0.86203 to 0.52592, saving model to /models


40/40 [==============================] - 59s 1s/step - loss: 0.5682 - accuracy: 0.7252 - val_loss: 0.5259 - val_accuracy: 0.7595 - lr: 0.0010
Epoch 3/100
40/40 [==============================] - ETA: 0s - loss: 0.5464 - accuracy: 0.7604
Epoch 3: val_loss did not improve from 0.52592
40/40 [==============================] - 57s 1s/step - loss: 0.5464 - accuracy: 0.7604 - val_loss: 0.5295 - val_accuracy: 0.7722 - lr: 0.0010
Epoch 4/100
40/40 [==============================] - ETA: 0s - loss: 0.5338 - accuracy: 0.7540
Epoch 4: val_loss improved from 0.52592 to 0.49075, saving model to /models


40/40 [==============================] - 61s 2s/step - loss: 0.5338 - accuracy: 0.7540 - val_loss: 0.4908 - val_accuracy: 0.7595 - lr: 0.0010
Epoch 5/100
40/40 [==============================] - ETA: 0s - loss: 0.5213 - accuracy: 0.7668
Epoch 5: val_loss improved from 0.49075 to 0.47710, saving model to /models


40/40 [==============================] - 106s 3s/step - loss: 0.5213 - accuracy: 0.7668 - val_loss: 0.4771 - val_accuracy: 0.7595 - lr: 0.0010
Epoch 6/100
40/40 [==============================] - ETA: 0s - loss: 0.5166 - accuracy: 0.7636
Epoch 6: val_loss did not improve from 0.47710
40/40 [==============================] - 9s 231ms/step - loss: 0.5166 - accuracy: 0.7636 - val_loss: 0.5374 - val_accuracy: 0.8481 - lr: 0.0010
Epoch 7/100
40/40 [==============================] - ETA: 0s - loss: 0.5024 - accuracy: 0.7700
Epoch 7: val_loss did not improve from 0.47710

Epoch 7: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
40/40 [==============================] - 57s 1s/step - loss: 0.5024 - accuracy: 0.7700 - val_loss: 0.4869 - val_accuracy: 0.8481 - lr: 0.0010
Epoch 8/100
40/40 [==============================] - ETA: 0s - loss: 0.4923 - accuracy: 0.7732
Epoch 8: val_loss improved from 0.47710 to 0.45395, saving model to /models


...

40/40 [==============================] - 63s 2s/step - loss: 0.4391 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-07
Epoch 65/100
40/40 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.7891
Epoch 65: val_loss improved from 0.40378 to 0.40378, saving model to /models



Epoch 65: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-08.
40/40 [==============================] - 106s 3s/step - loss: 0.4391 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-07
Epoch 66/100
40/40 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.7891
Epoch 66: val_loss improved from 0.40378 to 0.40378, saving model to /models


40/40 [==============================] - 63s 2s/step - loss: 0.4391 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-08
Epoch 67/100
40/40 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.7891
Epoch 67: val_loss improved from 0.40378 to 0.40378, saving model to /models



Epoch 67: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-09.
40/40 [==============================] - 107s 3s/step - loss: 0.4391 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-08
Epoch 68/100
40/40 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.7891
Epoch 68: val_loss did not improve from 0.40378
40/40 [==============================] - 10s 247ms/step - loss: 0.4391 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-09
Epoch 69/100
40/40 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.7891
Epoch 69: val_loss did not improve from 0.40378

Epoch 69: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.
40/40 [==============================] - 57s 1s/step - loss: 0.4391 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-09
Epoch 70/100
40/40 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.7891
Epoch 70: val_loss

40/40 [==============================] - 64s 2s/step - loss: 0.4391 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-10
Epoch 71/100
40/40 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.7891
Epoch 71: val_loss did not improve from 0.40378

Epoch 71: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-11.
40/40 [==============================] - 58s 1s/step - loss: 0.4391 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-10
Epoch 72/100
40/40 [==============================] - ETA: 0s - loss: 0.4390 - accuracy: 0.7891
Epoch 72: val_loss did not improve from 0.40378
40/40 [==============================] - 9s 231ms/step - loss: 0.4390 - accuracy: 0.7891 - val_loss: 0.4038 - val_accuracy: 0.8354 - lr: 1.0000e-11
Epoch 73/100
40/40 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.7891
Epoch 73: val_loss did not improve from 0.40378

Epoch 73: ReduceLROnPlateau reducing learning rat